In [1]:
!pip install tiktoken

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for tiktoken from https://files.pythonhosted.org/packages/1e/86/eea2309dc258fb86c7d9b10db536434fc16420feaa3b6113df18b23db7c2/tiktoken-0.8.0-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/884.5 kB ? eta -:--:--
   ---------------------------------------- 0.0/884.5 kB ? eta -:--:--
   ---------------------------------------- 10.2/884.5 kB ? eta -:--:--
   - ------------------------------------- 30.7/884.5 kB 435.7 kB/s eta 0:00:02
   -- ------------------------------------ 61.4/884.5 kB 465.5 kB/s eta 0:00:02
   ---- --------------------------------- 112.6/884.5 kB 652.2 kB/s eta 0:00:02
   ------- ------------------------------ 174.1/884.5 kB 748.1 kB/s eta 0:00:01
   ---------- --------------------------- 245.8/884.5 kB 942.1 kB/s eta 0:00:01
   ---------------- ----------------------- 368.6/884.5 kB 1.1 MB/s eta 0:00:01
   ---------------

DEPRECATION: Loading egg at c:\program files\python311\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..

[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## Input Target Pairs

In [1]:
import tiktoken

In [2]:
with open("the-verdict.txt","r",encoding="utf-8") as f:
    raw_text=f.read()

In [3]:
tokenizer=tiktoken.get_encoding("gpt2")

In [4]:
enc_text=tokenizer.encode(raw_text)

In [5]:
print(len(enc_text))

5145


In [6]:
enc_sample=enc_text[50:]

## Context Size

In [7]:
context_size=4 #length of input

x=enc_sample[:context_size]
y=enc_sample[1:context_size+1]

print("x:->",x)
print("y:->     ",y)

x:-> [290, 4920, 2241, 287]
y:->      [4920, 2241, 287, 257]


In [8]:
# Same thing as above
for i in range(1,context_size+1):
    context=enc_sample[:i]
    desired=enc_sample[i]
    print(context,"->",desired)

[290] -> 4920
[290, 4920] -> 2241
[290, 4920, 2241] -> 287
[290, 4920, 2241, 287] -> 257


In [9]:
for i in range(1,context_size+1):
    context=enc_sample[:i]
    desired=enc_sample[i]
    print(tokenizer.decode(context),"->",tokenizer.decode([desired]))

 and ->  established
 and established ->  himself
 and established himself ->  in
 and established himself in ->  a


In [10]:
from torch.utils.data import Dataset, DataLoader

class GPTDataset():
    def __init__(self,txt,tokenizer,max_length,stride):
        self.input_ids=[]
        self.target_ids=[]

        token_ids=tokenizer.encode(txt, allowed_special={"<|endoftext|>"})

        for i in range(0,len(token_ids)-max_length, stride):
            input_chunk=token_ids[i:i+max_length]
            target_chunk=token_ids[i+1:i+max_length+1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))
    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]
        

## Creating Dataloader

In [11]:
def create_dataloader(txt, batch_size=4, max_length=256, stride=128, shuffle=True, drop_last=True,num_workers=0):

    #Initialise Tokenizer
    tokenizer=tiktoken.get_encoding("gpt2")

    #Create Dataset
    dataset=GPTDataset(txt, tokenizer, max_length, stride)

    dataloader=DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )

    return dataloader

In [12]:
import torch

dataloader=create_dataloader(raw_text, batch_size=1,max_length=4,stride=1, shuffle=False)

data_iter= iter(dataloader)
first_batch=next(data_iter)
print(first_batch)

[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]


In [13]:
second_batch=next(data_iter)
print(second_batch)

[tensor([[ 367, 2885, 1464, 1807]]), tensor([[2885, 1464, 1807, 3619]])]


In [14]:
import torch

dataloader=create_dataloader(raw_text, batch_size=8,max_length=4,stride=4, shuffle=False)

data_iter= iter(dataloader)
input,target=next(data_iter)
print("Input:\n",input)
print("\nTarget:\n",target)

Input:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Target:
 tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])
